In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
device = 2
os.environ["CUDA_VISIBLE_DEVICES"]= str(device)
import torch 
import torch.nn as nn
device = torch.device("cuda:" + str(device) if torch.cuda.is_available() else "cpu")

import pandas as pd
import time
from multiprocessing import Pool
import os
import re
import numpy as np
import pycld2 as cld2  # language recognition 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from collections import Counter
# Let Jupyter Notebook show more content under cell
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [5]:
import pandas as pd

## 1. Read in the raw unlabeled data

In [6]:
df = pd.read_json('/shared/3/projects/benlitterer/podcastData/processed/floydMonth/floydMonthData.jsonl', orient="records", lines=True)

In [7]:
df.head(1)

,potentialOutPath,transcript,wCount,rssUrl,epTitle,epDescription,duration,pubDate,copyright,itunes:type,itunes:complete,guid,itunes:explicit,enclosure,itunes:image,transDict,id,title,lastUpdate,link,lastHttpStatus,dead,contentType,itunesId,originalUrl,itunesAuthor,itunesOwnerName,explicit,imageUrl,itunesType,generator,newestItemPubdate,language,oldestItemPubdate,episodeCount,popularityScore,priority,createdOn,updateFrequency,chash,host,newestEnclosureUrl,podcastGuid,podDescription,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,newestEnclosureDuration,oldestItemDatetime,cleanDates,cleanDatesLoc,parsedDuration,hours,transEnts,transStarts,transEnds,transTypes,descEnts,descStarts,descEnds,descTypes,epDescEnts,epDescStarts,epDescEnds,epDescTypes
0,/www.buzzsprout.com/3y/httpswww.buzzsprout.com7830203983942todayisyesterday.mp3MERGED,"n Shapiro and this is Sing Out Speak Out. Here I give you the philosophies, ideas and songs I've been working on for many years but until now I've shared just too few of because I've been afraid of you and worse I've been afraid of me. My will to connect with you and share the best of me has grown greater than any fear and that's why I am compelled to Sing Out Speak Out. Well hello everyone and welcome to Sing Out Speak Out episode number 20. I know crazy, it's going fast this crazy year that we are having 2020. I know for a lot of people the hard lockdown and the restrictions due to COVID-19 are still in full force and in Australia we are very lucky that we've managed to squash that curve for now due I think in part because we have a smaller population and we had advanced warning of what was happening in other countries so I think that held us in good stead but we're not over it yet of course and we're not gloating in any way whatsoever. But I know it can be tough when you're stuck in the same situation even if you might still have a job but you're working from home and one day you're homeschooling the next day you're doing your job at home and next day it's the weekend but it doesn't really feel like the weekend because you can't go anywhere. Every day is a bit the same. It's like Groundhog Day basically. So as I'm introducing songs to the Kiss Tone album Way to Nowhere which I made with my great friends Mike Haveername and Lee Walker. I'm going to skip to track number four because it's called Today is Yesterday. I'm not going to say too much about the song I'll let the song do the talking but it was a lot of fun to record even though the message itself is kind of serious. All the horns and the strings you hear are played by real musicians all of those layered vocals we did together and just vocal on top of vocal and a lot of work went into this song it was a heap of fun. It's one of my favourite bass parts that I've ever recorded and thanks to our producer Lindsey Rhimes who was there for every note guiding us getting the best out of us when we recorded this song. I hope you enjoy it. Without further adieu this is track four from our album Kiss Tone album Way to Nowhere. The song is called Today is Yesterday. I hope you have a great week. I'll be back next week with another episode of Sing Out Speak Out. [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] [Music] You'll expend the restore Jump slower on the same reflection If you're never gonna change direction Today is yesterday over, over again This we wanted play over, in, in the rest Got to turn your crazy world around So that I don't own your way to nowhere now Today is yesterday Press room on and press the line Today is yesterday You've got to turn your crazy world around So that I don't own your way So that I don't own your way You've got to turn your crazy world around So that I don't own your way Today is yesterday You've got t

In [5]:
soted_dates = sorted(df['pubDate'])

In [17]:
df.columns

Index(['potentialOutPath', 'transcript', 'wCount', 'rssUrl', 'epTitle',
       'epDescription', 'duration', 'pubDate', 'copyright', 'itunes:type',
       'itunes:complete', 'guid', 'itunes:explicit', 'enclosure',
       'itunes:image', 'transDict', 'id', 'title', 'lastUpdate', 'link',
       'lastHttpStatus', 'dead', 'contentType', 'itunesId', 'originalUrl',
       'itunesAuthor', 'itunesOwnerName', 'explicit', 'imageUrl', 'itunesType',
       'generator', 'newestItemPubdate', 'language', 'oldestItemPubdate',
       'episodeCount', 'popularityScore', 'priority', 'createdOn',
       'updateFrequency', 'chash', 'host', 'newestEnclosureUrl', 'podcastGuid',
       'podDescription', 'category1', 'category2', 'category3', 'category4',
       'category5', 'category6', 'category7', 'category8', 'category9',
       'category10', 'newestEnclosureDuration', 'oldestItemDatetime',
       'cleanDates', 'cleanDatesLoc', 'parsedDuration', 'hours', 'transEnts',
       'transStarts', 'transEnds', 'trans

In [ ]:
df.shape

## 2. Labeling strategy: 
### - A podcast is "politics" if it only has one single category assigned, and that being "politics". If none of its 10 category has "politics", it's non-politics
### - As we rely on weakly labeled data, this can keep our labeled data as clean as possible


In [25]:
def label_pol(row):
    if str(row['category1']).lower() == 'politics' and all(pd.isnull(row[col]) for col in ['category2', 'category3', 'category4','category5','category6','category7','category8','category9', 'category10']):
        return 1
    elif not any('politics' in str(category).lower() for category in row[['category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10']]):
        return 0
    else:
        return 2

In [ ]:
pd.options.mode.chained_assignment = None
df['is_politics'] = df.apply(label_pol, axis=1)

In [ ]:
df[df["is_politics"]==0].shape

In [ ]:
df[df["is_politics"]==1].shape

## The first method yielded too limited positive samples. Thus, we label a podcast as politics as long as the top 1 category is "politics"

In [8]:
def label_pol_new(row):
    if str(row['category1']).lower() == 'politics':
        return 1
    elif not any('politics' in str(category).lower() for category in row[['category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10']]):
        return 0
    else:
        return 2

In [9]:
pd.options.mode.chained_assignment = None
df['is_politics_new'] = df.apply(label_pol_new, axis=1)

In [13]:
df[df["is_politics_new"]==1].shape

(22, 74)

## Also too limited data. Try the third strategy. We label a podcast as politics if its top 3 category includes "politics"

In [10]:
def label_pol_third(row):
    if str(row['category1']).lower() == 'politics' or str(row['category2']).lower() == 'politics' or str(row['category3']).lower() == 'politics':
        return 1
    elif not any('politics' in str(category).lower() for category in row[['category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10']]):
        return 0
    else:
        return 2

In [ ]:
pd.options.mode.chained_assignment = None
df['is_politics_top_3'] = df.apply(label_pol_third, axis=1)

In [12]:
df.columns

Index(['potentialOutPath', 'transcript', 'wCount', 'rssUrl', 'epTitle',
       'epDescription', 'duration', 'pubDate', 'copyright', 'itunes:type',
       'itunes:complete', 'guid', 'itunes:explicit', 'enclosure',
       'itunes:image', 'transDict', 'id', 'title', 'lastUpdate', 'link',
       'lastHttpStatus', 'dead', 'contentType', 'itunesId', 'originalUrl',
       'itunesAuthor', 'itunesOwnerName', 'explicit', 'imageUrl', 'itunesType',
       'generator', 'newestItemPubdate', 'language', 'oldestItemPubdate',
       'episodeCount', 'popularityScore', 'priority', 'createdOn',
       'updateFrequency', 'chash', 'host', 'newestEnclosureUrl', 'podcastGuid',
       'podDescription', 'category1', 'category2', 'category3', 'category4',
       'category5', 'category6', 'category7', 'category8', 'category9',
       'category10', 'newestEnclosureDuration', 'oldestItemDatetime',
       'cleanDates', 'cleanDatesLoc', 'parsedDuration', 'hours', 'transEnts',
       'transStarts', 'transEnds', 'trans

In [14]:
df[df["is_politics_top_3"]==1].shape

(7792, 74)

## By using this strategy, we have 7792 transcripts as "politics". That's enough data to start with

In [15]:
df_pol = df[df["is_politics_top_3"]==1].copy()

In [16]:
df_pol.to_csv("pol_trans.csv", index=False)